In [1]:
## STANDARD PYTHON LIBS
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## ADDING TO ROOT
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(ROOT_DIR)

## CUSTOM LIBS
import src.utils as utils
import src.model as modellib
from src.config import Config
import src.visualize as visualize

# GPU LIBS
import keras
import tensorflow as tf

cuda_version = os.popen("cat /usr/local/cuda/version.txt ").read()
print ('TF : ', tf.__version__, '  Keras : ', keras.__version__, '  CUDA : ', cuda_version)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


TF :  1.6.0   Keras :  2.1.5   CUDA :  CUDA Version 9.0.176



In [ ]:
if utils.check_gpu(verbose=0):
    pass
    #import keras.backend as K
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True 
    #K.tensorflow_backend.set_session(tf.Session(config=config))
else:
    sys.exit(1)

In [2]:
if 'src.mapillary' in sys.modules : del sys.modules['src.mapillary']
if 'src.utils'     in sys.modules : del sys.modules['src.utils']
if 'src.model'     in sys.modules : del sys.modules['src.model']

import src.utils as utils
import src.model as modellib
import src.mapillary as mapillary

if __name__ == "__main__":
#     url_dataset = '/home/play/datasets/open_datasets/mapillary'
#     mapillary_mapper = '/home/play/playment/Mask_RCNN/demo/raw/merge__cityscapes_mapillary_v2.json'
    mapillary_config = mapillary.MapillaryConfig(images_per_gpu=16, gpu_count = 1)
    mapillary_config.STEPS_PER_EPOCH  = 2250 #2250
    mapillary_config.VALIDATION_STEPS = 50 #50
    print (mapillary_config.BATCH_SIZE)
    trainData = '/home/ubuntu/datasets/open_datasets/mapillary_hdf5_16/mapillary-vistas-dataset_public_v1.0/training'
    valData   = '/home/ubuntu/datasets/open_datasets/mapillary_hdf5_16/mapillary-vistas-dataset_public_v1.0/validation'
    
#     trainData = mapillary.MapillaryDataset(url_dataset, mapillary_mapper, mapillary_config, data_type = 'train')
#     valData   = mapillary.MapillaryDataset(url_dataset, mapillary_mapper, mapillary_config, data_type = 'val')

16


In [ ]:
# idx, show, verbose, test = 2459, True, True, True
# idx, show, verbose, test = 12488, True, True, True
# idx, show, verbose, test = 0, False, False, False
# img = trainData.load_image(idx, show=show)
# masks, class_ids = trainData.load_mask(idx, show=show, verbose=verbose, test=test)

## TRAINING

In [3]:
if 'src.model' in sys.modules : del sys.modules['src.model']
if 'src.utils'     in sys.modules : del sys.modules['src.utils']
import src.utils as utils
import src.model as modellib

    
MODEL_DIR = os.path.join(ROOT_DIR, 'demo', 'model', 'logs')
COCO_MODEL_PATH = os.path.join(ROOT_DIR, 'demo', 'model', "mask_rcnn_coco.h5")
model = modellib.MaskRCNN(mode="training", config=mapillary_config, model_dir=MODEL_DIR)

# init_with = "coco"  # imagenet, coco, or last
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model_path = model.find_last()[1]
    print (' - Path : ', model_path)
    model.load_weights(model_path, by_name=True)

 - Path :  /home/ubuntu/playment/Mask_RCNN/demo/model/logs/mapillary20180315T0317/mask_rcnn_mapillary_0173.h5


In [ ]:
if os.path.exists(trainData) & os.path.exists(valData): 
    model.train(trainData, valData, 
                learning_rate=mapillary_config.LEARNING_RATE
                , epochs=250
                , layers='heads'
                , use_multiprocessing = True
                , use_workers = 2 
                , verbose_generator = False
                
               )
else:
    print ('os.path.exists(trainData) : ', os.path.exists(trainData))
    print ('os.path.exists(valData)   : ', os.path.exists(valData))


Starting at epoch 174. LR=0.001

Checkpoint Path: /home/ubuntu/playment/Mask_RCNN/demo/model/logs/mapillary20180315T0317/mask_rcnn_mapillary_{epoch:04d}.h5

Total params: 63,803,372
Trainable params: 21,139,052
Non-trainable params: 42,664,320

Compiled and using 2 workers! 



/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 175/250
2250/2250 [==============================] - 2097s 932ms/step - loss: 1.2445 - rpn_class_loss: 0.0569 - rpn_bbox_loss: 0.6436 - mrcnn_class_loss: 0.1296 - mrcnn_bbox_loss: 0.2021 - mrcnn_mask_loss: 0.2122 - val_loss: 1.9389 - val_rpn_class_loss: 0.0667 - val_rpn_bbox_loss: 1.0548 - val_mrcnn_class_loss: 0.3227 - val_mrcnn_bbox_loss: 0.2510 - val_mrcnn_mask_loss: 0.2437
Epoch 176/250
2250/2250 [==============================] - 2142s 952ms/step - loss: 1.2332 - rpn_class_loss: 0.0565 - rpn_bbox_loss: 0.6353 - mrcnn_class_loss: 0.1280 - mrcnn_bbox_loss: 0.2013 - mrcnn_mask_loss: 0.2121 - val_loss: 1.9392 - val_rpn_class_loss: 0.0667 - val_rpn_bbox_loss: 1.1017 - val_mrcnn_class_loss: 0.2642 - val_mrcnn_bbox_loss: 0.2722 - val_mrcnn_mask_loss: 0.2343
Epoch 177/250
2250/2250 [==============================] - 2154s 957ms/step - loss: 1.2173 - rpn_class_loss: 0.0560 - rpn_bbox_loss: 0.6245 - mrcnn_class_loss: 0.1267 - mrcnn_bbox_loss: 0.1990 - mrcnn_mask_loss: 0.2110 - val_los

2250/2250 [==============================] - 2146s 954ms/step - loss: 1.0366 - rpn_class_loss: 0.0514 - rpn_bbox_loss: 0.4942 - mrcnn_class_loss: 0.1104 - mrcnn_bbox_loss: 0.1762 - mrcnn_mask_loss: 0.2045 - val_loss: 2.3529 - val_rpn_class_loss: 0.0724 - val_rpn_bbox_loss: 1.3818 - val_mrcnn_class_loss: 0.4074 - val_mrcnn_bbox_loss: 0.2639 - val_mrcnn_mask_loss: 0.2274
Epoch 197/250
2250/2250 [==============================] - 2104s 935ms/step - loss: 1.0344 - rpn_class_loss: 0.0513 - rpn_bbox_loss: 0.4937 - mrcnn_class_loss: 0.1094 - mrcnn_bbox_loss: 0.1757 - mrcnn_mask_loss: 0.2043 - val_loss: 2.1622 - val_rpn_class_loss: 0.0633 - val_rpn_bbox_loss: 1.0882 - val_mrcnn_class_loss: 0.5216 - val_mrcnn_bbox_loss: 0.2556 - val_mrcnn_mask_loss: 0.2335
Epoch 198/250
2250/2250 [==============================] - 2197s 976ms/step - loss: 1.0279 - rpn_class_loss: 0.0512 - rpn_bbox_loss: 0.4874 - mrcnn_class_loss: 0.1092 - mrcnn_bbox_loss: 0.1757 - mrcnn_mask_loss: 0.2043 - val_loss: 2.0965 - va

In [ ]:
sys.exit(1)

# INFERENCE

In [ ]:
if 'src.model' in sys.modules : del sys.modules['src.model']
import src.model as modellib

MODEL_DIR = os.path.join(ROOT_DIR, 'demo', 'model', 'logs')

class InferenceConfig(mapillary.MapillaryConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
print (' - Batch Size : ', inference_config.BATCH_SIZE)
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
model_path = model.find_last()[1]
print (' - Model Path : ', model_path)

if model_path != None:
    model.load_weights(model_path, by_name=True)
else:
    sys.exit(1)

In [ ]:
if 'src.mapillary' in sys.modules : del sys.modules['src.mapillary']
import src.mapillary as mapillary
testData = mapillary.MapillaryDataset(url_dataset, mapillary_mapper, mapillary_config, data_type = 'test')

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(15,15))
    return ax

test_img = testData.load_image(1586)
results  = model.detect([test_img], verbose=1)
r = results[0]

if 'src.visualize' in sys.modules : del sys.modules['src.visualize']
import src.visualize as visualize
visualize.display_instances(test_img, r['rois'], r['masks'], r['class_ids'], 
                            testData.class_names, r['scores'], ax=get_ax())

# SCRATCHPAD